<a align="center" href="https://ultralytics.com/hub" target="_blank">
<img width="1024", src="https://github.com/ultralytics/assets/raw/main/im/ultralytics-hub.png"></a>

<div align="center">

[中文](https://docs.ultralytics.com/zh/hub/) | [한국어](https://docs.ultralytics.com/ko/hub/) | [日本語](https://docs.ultralytics.com/ja/hub/) | [Русский](https://docs.ultralytics.com/ru/hub/) | [Deutsch](https://docs.ultralytics.com/de/hub/) | [Français](https://docs.ultralytics.com/fr/hub/) | [Español](https://docs.ultralytics.com/es/hub/) | [Português](https://docs.ultralytics.com/pt/hub/) | [Türkçe](https://docs.ultralytics.com/tr/hub/) | [Tiếng Việt](https://docs.ultralytics.com/vi/hub/) | [العربية](https://docs.ultralytics.com/ar/hub/)

  <a href="https://github.com/ultralytics/hub/actions/workflows/ci.yaml"><img src="https://github.com/ultralytics/hub/actions/workflows/ci.yaml/badge.svg" alt="CI CPU"></a>
  <a href="https://colab.research.google.com/github/ultralytics/hub/blob/main/hub.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>

Welcome to the [Ultralytics](https://ultralytics.com/) HUB notebook!

This notebook allows you to train Ultralytics [YOLO](https://github.com/ultralytics/ultralytics) 🚀 models using [HUB](https://hub.ultralytics.com/). Please browse the HUB <a href="https://docs.ultralytics.com/hub/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/hub/issues/new/choose">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!
</div>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [3]:
%pip install ultralytics  # install
from ultralytics import YOLO, checks, hub
from ultralytics.utils.files import WorkingDirectory

checks()  # checks

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.3/235.7 GB disk)


# Start

⚡ Login with your API key, load your YOLO 🚀 model, and start training in 3 lines of code!

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


import shutil
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywh
# Download
dir = Path('/content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K')  # dataset root dir
parent = Path(dir.parent)  # download dir
urls = ['http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz']
if not parent.exists():
    os.mkdirs(parent)
download(urls, dir=parent)

# Rename directories
if dir.exists():
    shutil.rmtree(dir)
(parent / 'SKU110K_fixed').rename(dir)  # rename dir
(dir / 'labels').mkdir(parents=True, exist_ok=True)  # create labels dir

# Convert labels
names = 'image', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height'  # column names
for d in 'annotations_train.csv', 'annotations_val.csv', 'annotations_test.csv':
    x = pd.read_csv(dir / 'annotations' / d, names=names).values  # annotations
    images, unique_images = x[:, 0], np.unique(x[:, 0])
    with open((dir / d).with_suffix('.txt').__str__().replace('annotations_', ''), 'w') as f:
        f.writelines(f'./images/{s}\n' for s in unique_images)
    for im in tqdm(unique_images, desc=f'Converting {dir / d}'):
        cls = 0  # single-class dataset
        with open((dir / 'labels' / im).with_suffix('.txt'), 'a') as f:
            for r in x[images == im]:
                w, h = r[6], r[7]  # image width, height
                xywh = xyxy2xywh(np.array([[r[1] / w, r[2] / h, r[3] / w, r[4] / h]]))[0]  # instance
                f.write(f"{cls} {xywh[0]:.5f} {xywh[1]:.5f} {xywh[2]:.5f} {xywh[3]:.5f}\n")  # write label


Mounted at /content/drive
Unzipping /content/drive/MyDrive/yolo/sku110k/datasets/SKU110K_fixed.tar.gz to /content/drive/MyDrive/yolo/sku110k/datasets...


Converting /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/annotations_train.csv:  20%|██        | 1645/8219 [01:24<04:21, 25.10it/s]

# Train

In [ ]:
hub.login('5a487d90ec3a393c881d25748a026ac80f08e64702')

model = YOLO('/content/drive/MyDrive/yolo/yolo11n.pt')
results = model.train(data='/content/drive/MyDrive/yolo/sku110k.yaml',batch=16)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
